# SIAA: Regresión lineal 

## Autores: Javier, Laura, Martín, Hugo, Raúl

En este notebook haremos un estudio paso por paso de los datos de los estudiantes de un curso de mátematicas.

Probaremos con distintos modelos, veremos cual se ajusta mejor y haremos una predicción.

Enlace del Dataset del curso de Matemáticas obtenido: https://www.kaggle.com/datasets/devansodariya/student-performance-data

Enlace con la explicación de las columnas: https://archive.ics.uci.edu/dataset/320/student+performance

#### Columnas de student-mat.csv (curso de Matemáticas):

**school** - escuela del estudiante (binario: "GP" - Gabriel Pereira o "MS" - Mousinho da Silveira)

**sex** - sexo del estudiante (binario: "F" - femenino o "M" - masculino)

**age** - edad del estudiante (numérico: de 15 a 22)

**address** - tipo de dirección del hogar del estudiante (binario: "U" - urbano o "R" - rural)

**famsize** - tamaño de la familia (binario: "LE3" - menor o igual a 3 o "GT3" - mayor que 3)

**Pstatus** - estado de convivencia de los padres (binario: "T" - juntos o "A" - separados)

**Medu** - nivel educativo de la madre (numérico: 0 - ninguno, 1 - educación primaria (4º grado), 2 - de 5º a 9º grado, 3 - educación secundaria o 4 - educación superior)

**Fedu** - nivel educativo del padre (numérico: 0 - ninguno, 1 - educación primaria (4º grado), 2 - de 5º a 9º grado, 3 - educación secundaria o 4 - educación superior)

**Mjob** - ocupación de la madre (nominal: "teacher", relacionado con "health" (salud), servicios civiles "services" (ej. administración o policía), "at_home" o "other")

**Fjob** - ocupación del padre (nominal: "teacher", relacionado con "health" (salud), servicios civiles "services" (ej. administración o policía), "at_home" o "other")

**reason** - motivo para elegir esta escuela (nominal: cercana a "home", "reputation" de la escuela, preferencia por el "course" o "other")

**guardian** - tutor del estudiante (nominal: "mother", "father" o "other")

**traveltime** - tiempo de viaje de casa a la escuela (numérico: 1 - <15 min., 2 - 15 a 30 min., 3 - 30 min. a 1 hora, o 4 - >1 hora)

**studytime** - tiempo de estudio semanal (numérico: 1 - <2 horas, 2 - 2 a 5 horas, 3 - 5 a 10 horas, o 4 - >10 horas)

**failures** - número de asignaturas reprobadas en el pasado (numérico: n si 1<=n<3, de lo contrario 4)

**schoolsup** - apoyo educativo adicional (binario: yes o no)

**famsup** - apoyo educativo por parte de la familia (binario: yes o no)

**paid** - clases particulares pagadas en la materia del curso (Matemáticas o Portugués) (binario: yes o no)

**activities** - participación en actividades extracurriculares (binario: yes o no)

**nursery** - asistencia a guardería (binario: yes o no)

**higher** - desea cursar educación superior (binario: yes o no)

**internet** - acceso a Internet en casa (binario: yes o no)

**romantic** - tiene una relación romántica (binario: yes o no)

**famrel** calidad de las relaciones familiares (numérico: de 1 - muy mala a 5 - excelente)

**freetime** tiempo libre después de la escuela (numérico: de 1 - muy bajo a 5 - muy alto)

**goout** - frecuencia de salidas con amigos (numérico: de 1 - muy baja a 5 - muy alta)

**Dalc** - consumo de alcohol en días laborales (numérico: de 1 - muy bajo a 5 - muy alto)

**Walc** - consumo de alcohol los fines de semana (numérico: de 1 - muy bajo a 5 - muy alto)

**health** - estado de salud actual (numérico: de 1 - muy malo a 5 - muy bueno)

**absences** - número de ausencias escolares (numérico: de 0 a 93)

#### Calificaciones del curso:

**G1** - calificación del primer período (numérico: de 0 a 20)

**G2** - calificación del segundo período (numérico: de 0 a 20)

**G3** - calificación final (numérico: de 0 a 20)

Importamos todas las librerías necesarias para realizar las regresiones lineales con diferentes modelos de la misma y las predicciones con hiperparámetros

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from scipy.stats import uniform

### Atributos categóricos:

El dataframe original tiene muchas columnas categóricas, les aplicaremos un OneHotEncoder para pasarlas a numéricas

In [2]:
# Cargar el dataset
dataset = pd.read_csv('student-mat.csv', delimiter=';')

# Función para aplicar el OneHotEncoder
def encoder(columna, df):
    encoder = OneHotEncoder()

    colegios_valores_encoded = encoder.fit_transform(df[[columna]])
    colegios_columnas_encoded = encoder.get_feature_names_out([columna])
    encoded_df = pd.DataFrame(colegios_valores_encoded.toarray(), columns=colegios_columnas_encoded)

    df = pd.concat([df, encoded_df], axis=1)
    df = df.drop(columns=[columna])
    return df

Podemos ver las columnas que son de tipo objeto (Strings)

In [3]:
# Vemos las columnas categóricas
columnas_obj = []

for columna in dataset.columns:
    if dataset[columna].dtype == object:
        columnas_obj.append(columna)

print(columnas_obj)

['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']


Aplicamos la función a todas las columnas categóricas

In [4]:
# Les aplicamos la función a cada columna categórica
for columna in columnas_obj:
    dataset = encoder(columna, dataset)

Escalamos los datos para que todos estén en un rango parecido con StandardScaler, que hace uso del Z-Score

In [5]:
# Aplicamos el StandardScaler
scaler = StandardScaler()
dataset_scaled = scaler.fit_transform(dataset)

dataset_scaled = pd.DataFrame(dataset_scaled, columns=dataset.columns)

### Correlaciones:

Aqui mostramos los coeficientes de correlación de G3 con las demás columnas

In [6]:
# Seleccionar las columnas para el modelo de regresión lineal: horas de estudio, edad, faltas, repeticiones, nota 1º, 2º y 3º evaluación
X = dataset_scaled[['studytime', 'age', 'absences', 'failures', 'goout', 'G1', 'G2']]
Y = dataset_scaled['G3']

# Mostrar correlaciones
plt.figure(figsize=(12, 8))
correlation_matrix = dataset_scaled.corr()
sorted_corr = correlation_matrix['G3'].sort_values(ascending=False)
print(sorted_corr)

# Dividir el dataset en conjunto de entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42)

G3                   1.000000
G2                   0.904868
G1                   0.801468
Medu                 0.217147
higher_yes           0.182465
Fedu                 0.152457
romantic_no          0.129970
Mjob_health          0.116158
address_U            0.105756
sex_M                0.103456
paid_yes             0.101996
internet_yes         0.098483
studytime            0.097820
reason_reputation    0.095692
Fjob_teacher         0.095374
schoolsup_no         0.082788
famsize_LE3          0.081407
Mjob_services        0.078429
Pstatus_A            0.058009
Mjob_teacher         0.057712
Fjob_health          0.057111
reason_other         0.052008
nursery_yes          0.051568
famrel               0.051363
school_GP            0.045017
famsup_no            0.039157
absences             0.034247
guardian_father      0.032493
guardian_mother      0.022338
activities_yes       0.016100
freetime             0.011307
Fjob_at_home        -0.013385
activities_no       -0.016100
Fjob_servi

<Figure size 1200x800 with 0 Axes>

### Regresión lineal:

In [7]:
# Entrenamos modelo
model = LinearRegression()
model.fit(X_train, Y_train)

# Realizamos predicción
Y_pred = model.predict(X_test)

# Obtenemos métricas
# MSE: Error
# R2: Comparación del valor obtenido de la predicción con el valor real
mse = mean_squared_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)
print(f'Linear Regression - MSE: {mse}, R2: {r2}')
coefficients = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])
print(coefficients)

Linear Regression - MSE: 0.22141371761557255, R2: 0.7739276873831538
           Coefficient
studytime    -0.006962
age          -0.051816
absences      0.076069
failures     -0.064376
goout         0.042127
G1            0.118637
G2            0.795763


Métricas: <br>
MSE: 0.2214<br>
R<sup>2</sup>: 0.7739<br>

R<sup>2</sup> bastante alto, el modelo es útil para predecir.

El tiempo de estudio y las faltas parecen tener muy poco peso en las calificaciones finales.

Parece que las notas bajan con la edad, cuanto más avanzada la edad, un poco peores las notas.

La primera evaluación influye un poco en la final y la segunda parece influír de forma significativa.

### Regularización

#### Lasso L1

In [8]:
lasso = Lasso(alpha=0.1)
lasso.fit(X_train.values, Y_train.values)
Y_pred_lasso = lasso.predict(X_test)
mse_lasso = mean_squared_error(Y_test, Y_pred_lasso)
r2_lasso = r2_score(Y_test, Y_pred_lasso)
print(f'Lasso Regression - MSE: {mse_lasso}, R2: {r2_lasso}')
coefficients_lasso = pd.DataFrame(lasso.coef_, X.columns, columns=['Coefficient'])
print(coefficients_lasso)

Lasso Regression - MSE: 0.20237279559358892, R2: 0.7933692347372375
           Coefficient
studytime     0.000000
age          -0.000000
absences      0.000000
failures     -0.000000
goout         0.000000
G1            0.051693
G2            0.768863


c:\Users\laura.mellaramirez\Documents\SIAA_prediccion_lineal_notas\.venv\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(


Métricas:<br>
MSE: 0.202<br>
R<sup>2</sup>: 0.793<br>

R<sup>2</sup> más alto que el LinearRegression, el modelo es mejor para predecir.

Se considera que el tiempo de estudio, las faltas, la edad y las salidas tienen poca o ninguna relación con las calificaciones finales.

Parece que las notas bajan con la edad, cuanto más avanzada la edad, un poco peores las notas.

La primera evaluación influye un poco en la final y la segunda parece influír de forma significativa.

#### Ridge L2

In [9]:
ridge = Ridge(alpha=0.1)
ridge.fit(X_train.values, Y_train.values)
Y_pred_ridge = ridge.predict(X_test)
mse_ridge = mean_squared_error(Y_test, Y_pred_ridge)
r2_ridge = r2_score(Y_test, Y_pred_ridge)
print(f'Ridge Regression - MSE: {mse_ridge}, R2: {r2_ridge}')
coefficients_ridge = pd.DataFrame(ridge.coef_, X.columns, columns=['Coefficient'])
print(coefficients_ridge)

Ridge Regression - MSE: 0.2213930552369067, R2: 0.7739487845029684
           Coefficient
studytime    -0.006966
age          -0.051859
absences      0.076051
failures     -0.064393
goout         0.042084
G1            0.119221
G2            0.795022


c:\Users\laura.mellaramirez\Documents\SIAA_prediccion_lineal_notas\.venv\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but Ridge was fitted without feature names
  warnings.warn(


Métricas:<br>
MSE: 0.221<br>
R<sup>2</sup>: 0.774

R<sup>2</sup> bastante alto, el modelo es útil para predecir.

El tiempo de estudio y las faltas parecen tener muy poco peso en las calificaciones finales.

Parece que las notas bajan con la edad, cuanto más avanzada la edad, un poco peores las notas.

La primera evaluación influye un poco en la final y la segunda parece influír de forma significativa.

### ElasticNet

In [10]:
elasticnet = ElasticNet(alpha=0.1, l1_ratio=0.5)
elasticnet.fit(X_train.values, Y_train.values)
Y_pred_elasticnet = elasticnet.predict(X_test)
mse_elasticnet = mean_squared_error(Y_test, Y_pred_elasticnet)
r2_elasticnet = r2_score(Y_test, Y_pred_elasticnet)
print(f'ElasticNet Regression - MSE: {mse_elasticnet}, R2: {r2_elasticnet}')
coefficients_elasticnet = pd.DataFrame(elasticnet.coef_, X.columns, columns=['Coefficient'])
print(coefficients_elasticnet)

ElasticNet Regression - MSE: 0.20563177005473823, R2: 0.7900416907118467
           Coefficient
studytime    -0.000000
age          -0.005769
absences      0.022251
failures     -0.039657
goout         0.000000
G1            0.148997
G2            0.691927


c:\Users\laura.mellaramirez\Documents\SIAA_prediccion_lineal_notas\.venv\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but ElasticNet was fitted without feature names
  warnings.warn(


Métricas: <br>
MSE: 0.205<br>
R<sup>2</sup>: 0.790<br>

R<sup>2</sup> bastante alto, el modelo es útil para predecir.

El tiempo de estudio y las faltas parecen tener muy poco peso en las calificaciones finales.

Parece que las notas bajan con la edad, cuanto más avanzada la edad, un poco peores las notas.

La primera evaluación influye un poco en la final y la segunda parece influír de forma significativa.

Vemos que el modelo que mejor se ajusta a la realidad es Lasso, por lo tanto es el elegido

### Hiperparámetros

Ajustamos Lasso con hiperparámetros

In [11]:
param_dist = {'alpha': uniform(0.1, 100)}
random_search = RandomizedSearchCV(lasso, param_distributions=param_dist, n_iter=100, cv=5, random_state=42)
random_search.fit(X_train.values, Y_train.values)

print("Mejores paramétros encontrados: ", random_search.best_params_)
random_search.predict(X_test)
print("Error cuadrático medio en prueba: ", mean_squared_error(Y_test, Y_pred))

Mejores paramétros encontrados:  {'alpha': np.float64(0.6522117123602399)}
Error cuadrático medio en prueba:  0.22141371761557255


c:\Users\laura.mellaramirez\Documents\SIAA_prediccion_lineal_notas\.venv\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(


Lasso tenía un error cuadrático medio de 0.202, por lo tanto NO mejora con el hiperparámetro

Predecimos

In [12]:
# Columnas usadas actuales ['studytime', 'age', 'absences', 'failures', 'goout', 'G1', 'G2']
lasso.predict([[1, 15, 1, 1, 1, 12, 14]])

array([11.37721327])

Predecimos que la nota final de un estudiante de 15 años que estudia menos de 2 horas a la semana, tiene una falta, repitió una vez, sacó un 12 en la 1º evaluación y un 14 en la 2º será de 11.38

## Conclusiones:

- Todos los modelos que utilizamos tienen un poder de predicción parecido, Lasso gana pero por muy poco.

- Los atributos que más influyen de mayor a menor en que suba la nota final de los estudiantes son:

    La nota de la segunda evaluación.

    La nota de la primera evaluación.
    
    Un nivel de educación alto de los padres.
    
    Si el estudiante no está en una relación.


- Los atributos que más influyen de mayor a menor en que baje la nota final de los estudiantes son:

    El tiempo que le lleva al estudiante llegar al centro.

    Estar en una relación.

    Salir muy a menudo.

    La mayor edad.

    Un nivel bajo de educación de los padres.

    Haber repetido.

    ## Problemas encontrados:

        Mapa de calor no legible por la cantidad de columnas.

        Al hacer el OneHotEncoder se generaban muchas columnas y salían demasiadas en la matriz de correlación
    
        Hicimos la correlación de las características solo con G3